In [124]:
import pandas as pd
import time
import numpy as np
from numba import jit
from sklearn.model_selection import train_test_split
from multiprocessing import *
from sklearn.model_selection import KFold
import gc
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [142]:
start = time.time()
train = pd.read_csv("/home/saurabhg/PuertoSergo/train.csv")
test = pd.read_csv("/home/saurabhg/PuertoSergo/test.csv")
print(time.time()-start)
 
y = train['target']
testid= test['id'].values

train.drop(['id','target'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

8.60738611221


In [143]:
### Drop calc
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
f_cats = [f for f in train.columns if "_cat" in f]
train = train.drop(unwanted, axis=1)  
test = test.drop(unwanted, axis=1)

### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
#train['ps_reg_03_square'] = train['ps_reg_03']*train['ps_reg_03']
#test['ps_reg_03_square'] = test['ps_reg_03']*test['ps_reg_03']
#train['ps_car_14_square'] = train['ps_car_14']*train['ps_car_14']
#test['ps_car_14_square'] = test['ps_car_14']*test['ps_car_14']
def recon(reg):
    integer = int(np.round((40*reg)**2)) 
    for a in range(32):
        if (integer - a) % 31 == 0:
            A = a
    M = (integer - A)//31
    return A, M
train['ps_reg_A'] = train['ps_reg_03'].apply(lambda x: recon(x)[0])
train['ps_reg_M'] = train['ps_reg_03'].apply(lambda x: recon(x)[1])
train['ps_reg_A'].replace(19,-1, inplace=True)
train['ps_reg_M'].replace(51,-1, inplace=True)
test['ps_reg_A'] = test['ps_reg_03'].apply(lambda x: recon(x)[0])
test['ps_reg_M'] = test['ps_reg_03'].apply(lambda x: recon(x)[1])
test['ps_reg_A'].replace(19,-1, inplace=True)
test['ps_reg_M'].replace(51,-1, inplace=True)


### Froza's baseline
### Froza's baseline
### Froza's baseline
### Froza's baseline

d_median = train.median(axis=0)
d_mean = train.mean(axis=0)
d_skew = train.skew(axis=0)
one_hot = {c: list(train[c].unique()) for c in train.columns if c not in ['id','target']}

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c: #standard arithmetic
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(np.int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(np.int)

#    for c in one_hot:
#        if len(one_hot[c])>2 and len(one_hot[c]) < 7:
#            for val in one_hot[c]:
#                df[c+'_oh_' + str(val)] = (df[c].values == val).astype(np.int)
    return df

def multi_transform(df):
    print('Init Shape: ', df.shape)
    p = Pool(cpu_count())
    df = p.map(transform_df, np.array_split(df, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    print('After Shape: ', df.shape)
    return df

train = multi_transform(train)
test = multi_transform(test)





('Init Shape: ', (595212, 39))
('After Shape: ', (595212, 97))
('Init Shape: ', (892816, 39))
('After Shape: ', (892816, 97))


In [144]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

def ginic(actual, pred):
    actual = np.asarray(actual) 
    n = len(actual)
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
    return giniSum / n
 
def gini_normalized(a, p):
    if p.ndim == 2:
        p = p[:,1] 
    return ginic(a, p) / ginic(a, a)
    

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

In [145]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [146]:
#f_cats = [f for f in train.columns if "_cat" in f]
for f in f_cats:
    if '_median_range' not in f:
        if '_mean_range' not in f:
            train[f + "_avg"], test[f + "_avg"] = target_encode(trn_series=train[f],
                                             tst_series=test[f],
                                             target=y,
                                             min_samples_leaf=200,
                                             smoothing=10,
                                             noise_level=0)

In [149]:
train = train.drop(f_cats,axis =1)
test = test.drop(f_cats,axis =1)


In [150]:
MAX_ROUNDS = 5000
OPTIMIZE_ROUNDS = True
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  
# Note: I set EARLY_STOPPING_ROUNDS high so that (when OPTIMIZE_ROUNDS is set)
#       I will get lots of information to make my own judgment.  You should probably
#       reduce EARLY_STOPPING_ROUNDS if you want to do actual early stopping.

In [151]:
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
imp_df = np.zeros((len(train.columns), K))
sub_preds = np.zeros((len(test),K))
np.random.seed(0)
params = {'eta': 0.025, 'max_depth': 5, 
          'subsample': 0.9, 'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

In [152]:
# Run CV

y_valid_pred = 0*y
y_test_pred = 0

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = train.iloc[train_index,:].copy(), train.iloc[test_index,:].copy()
    X_test = test.copy()
    print( "\nFold ", i)
    
    # Enocode data
#     for f in f_cats:
#         X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
#                                                         trn_series=X_train[f],
#                                                         val_series=X_valid[f],
#                                                         tst_series=X_test[f],
#                                                         target=y_train,
#                                                         min_samples_leaf=200,
#                                                         smoothing=10,
#                                                         noise_level=0
#                                                         )
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_valid, y_valid), 'valid')]
        fit_model =  xgb.train(params, (xgb.DMatrix(X_train, y_train)), 5000,  watchlist, feval=gini_xgb, maximize=True, 
                  verbose_eval=100, early_stopping_rounds=70)
        print( "  Best N trees = ", fit_model.best_ntree_limit )
        print( "  Best gini = ", fit_model.best_score )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # Generate validation predictions for this fold
    pred = fit_model.predict(xgb.DMatrix(X_valid),ntree_limit=model.best_ntree_limit)
    print( "  Gini = ", gini_normalized(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    #Save Importance
#    imp_df[:, i] = fit_model.feature_importances_
    
    # Accumulate test set predictions
    sub_preds[:, i] = fit_model.predict(xgb.DMatrix(test),ntree_limit=model.best_ntree_limit)
    
    del X_test, X_train, X_valid, y_train
    


print( "\nGini for full training set:" )
gini_normalized(y, y_valid_pred)


('\nFold ', 0)
[0]	train-gini:0.179337	valid-gini:0.16451
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 70 rounds.
[100]	train-gini:0.26766	valid-gini:0.252872
[200]	train-gini:0.298487	valid-gini:0.272516
[300]	train-gini:0.31563	valid-gini:0.280699
[400]	train-gini:0.328155	valid-gini:0.284984
[500]	train-gini:0.339092	valid-gini:0.286473
[600]	train-gini:0.348639	valid-gini:0.286924
[700]	train-gini:0.35796	valid-gini:0.287212
[800]	train-gini:0.365665	valid-gini:0.287049
Stopping. Best iteration:
[748]	train-gini:0.361872	valid-gini:0.287239

('  Best N trees = ', 749)
('  Best gini = ', 0.287239)
('  Gini = ', 0.28708918141400053)
('\nFold ', 1)
[0]	train-gini:0.177056	valid-gini:0.16768
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 70 rounds.
[100]	train-gini:0.269748	valid-gini:0.252633
[200]	train-gini:0.

0.28637063462587992

In [153]:
sub_pred= np.delete(sub_preds,[2],axis =1)

In [154]:
import scipy
sub = pd.DataFrame()
sub['id'] = testid
sub["target"] = scipy.stats.hmean(sub_pred,axis=1)
sub.to_csv("submission_10312017_xgb_0.2847_with_target_encoding.csv", index=False, float_format="%.9f")